In [2]:
%load_ext autoreload
%autoreload 2

In [9]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
from pulp import *
from copy import deepcopy
while "notebooks" in os.getcwd():
    os.chdir("..")

from src.preprocessing.parser import Parser
from src.preprocessing.preprocessor import Preprocessor
from src.solvers.solution import Solution
from src.solvers.pulp_solver import PuLPSolver

from tqdm import tqdm
from typing import Dict

In [208]:
parser = Parser("data/testfiles/test1.txt")

In [209]:

info = parser.read()

28it [00:00, 50906.16it/s]


In [210]:
dataset : Dict[int, pd.DataFrame ]= info['data']
p, K, M, N = info['p'], info['K'], info['M'], info['N']

In [211]:
preprocessor = Preprocessor(
    K, 
    M,
    N,
    p,
    dataset
)

data_1 = preprocessor.remove_trivial_values()
data_2 = preprocessor.remove_ip_dominated(dataset)
data_3 = preprocessor.remove_lp_dominated(data_2)

100%|██████████| 4/4 [00:00<00:00, 4525.82it/s]


In [213]:

memo = dict()

def R(
    N, 
    p_max, 
    dataset : Dict[int, pd.DataFrame]
):
    
    global memo

    for idx, row in dataset[0].iterrows():
        
        k, m= row['k'], row['m']
        r_kmn, p_kmn = row['r_k,m,n'], row['p_k,m,n']
        
        if p_kmn <= p_max: 

            memo[tuple((0, p_kmn))] = r_kmn

    
    for n in range(1,N):
        new_values = {}
        for idx, row in dataset[n].iterrows():
            
            r_kmn = row['r_k,m,n']
            p_kmn = row['p_k,m,n']

            for (n,p), r in memo.items(): 

                if p + p_kmn > p_max:
                    continue

                key = tuple((n-1, p + p_kmn))

                if new_values.get(key) is not None:
                    new_values[key] = max(r + r_kmn, new_values[key])
                
                else:
                    new_values[key] = r + r_kmn
        memo = new_values       

    return max(memo.values())    

In [214]:
memo = {}
R(N, p, data_3)

365.0

In [215]:
items = {}
def P(
    N, 
    p_max,
    U, 
    dataset : Dict[int, pd.DataFrame]
):
    if U < 0:
        return -np.inf, 0
    global items

    for idx, row in dataset[0].iterrows():
        
        r_kmn, p_kmn = row['r_k,m,n'], row['p_k,m,n']

        key = tuple((0, r_kmn))
        if r_kmn <= U:
            items[key] = p_kmn

    for n in range(1,N):
        new_items = {}
        for idx, row in dataset[n].iterrows():
        
            r_kmn = row['r_k,m,n']
            p_kmn = row['p_k,m,n']

            for key, p in items.items():

                r = key[1]

                if r + r_kmn <= U:
                    t = tuple((n, r + r_kmn))
                    
                    # if there exists another n, (r+r_kmn) key

                    if new_items.get(t) is not None:
                        new_items[t] = min(p + p_kmn, new_items[t])
                    else:
                        new_items[t] = p + p_kmn
        items = new_items
    
    # return maximal r
    r_max = -1
    for key, p in items.items():
        if key[0] != N-1:
            continue

        r = key[1]
        if r> r_max and p <= p_max:
            r_max = r 
        
    return r_max

        

In [216]:
solver = PuLPSolver(
    K,
    M,
    N,
    p,
    data_3
)

solver.solve()
U = int(solver.solution.objective.value())

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /home/pedro/anaconda3/envs/user-scheduling-in-5g/lib/python3.12/site-packages/pulp/solverdir/cbc/linux/64/cbc /tmp/657814b442904cf08b529c34f1d6e88b-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /tmp/657814b442904cf08b529c34f1d6e88b-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 10 COLUMNS
At line 35 RHS
At line 41 BOUNDS
At line 50 ENDATA
Problem MODEL has 5 rows, 8 columns and 16 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 0 (-5) rows, 0 (-8) columns and 0 (-16) elements
Empty problem - 0 rows, 0 columns and 0 elements
Optimal - objective value 365
After Postsolve, objective 365, infeasibilities - dual 191 (4), primal 0 (0)
Presolved model was optimal, full model needs cleaning up
0  Obj 365 Dual inf 234.9072 (4)
4  Obj 365
Optimal - objective value 365
Optimal objective 365 - 4 i

In [217]:
U

365

In [218]:
P(N, p, U, data_3)

365.0

In [152]:
items[tuple((N-1, 1637))]

1000.0